In [1]:
#import libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import folium #for maps
import json # library to handle JSON files

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         202 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



certifi-2020.12.5    | 143 KB    | ####

Get data from wiki:
take html page, cit out table with postal code data
fit it first into list and then into the dataframe.

In [2]:
#get data from wiki as html page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

#cut out postalcode table from html page
startTable=results.text.find('<table class="wikitable sortable">\n') #where data i need starts
EndTable=results.text.find('</table>') #where data i need ends
tableData=results.text[startTable:EndTable]

#get column header names
tHeaders=tableData[tableData.find('<tr>\n<th>'):tableData.find('</th></tr>')]
tHeaders=tHeaders.replace('<tr>','').replace('</th>','').replace("\n", "")
hList=tHeaders.split('<th>')
hList.pop(0)

#get table data into list
tData=tableData[tableData.find('</th></tr>\n')+11:tableData.find('</tr></tbody>')+5]
tList=tData.replace("\n", "").split('</tr>')
i=0
#dData=pd.DataFrame
for n in tList:
    tList[i]=tList[i].replace('<tr>','')
    tList[i]=tList[i].replace('</td>','')
    tList[i]=tList[i].split('<td>')
    tList[i].pop(0)
    i=i+1
tList = list(filter(None, tList))

# create dataframe
dData=pd.DataFrame(tList)
dData.columns=hList

clean data :
remove not assigned borough rows as we dont analyse them
check not assigned neighborough to assign them borough name

In [3]:
#remove not assigned postal code to Borough
dData = dData[dData.Borough != 'Not assigned']

if (dData[dData.Neighbourhood == 'Not assigned'].shape[0]==0): 
    print('no need to assign borough names to neighborough ')
dData.set_index('Postal Code',inplace=True)
dData=dData.reset_index()
#dData.tail()

no need to assign borough names to neighborough 


In [4]:
#get size of dataframe
print ('size of dataframe is:', dData.shape)

size of dataframe is: (103, 3)


In [5]:
#get csv file because I couldnot get geodata from geocoder

!wget -q -O 'toronto_data.csv' 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
print('Data downloaded!')

Data downloaded!


In [6]:
geocsv_df = pd.read_csv('toronto_data.csv')
geocsv_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
#geocsv_df.head(2)

In [7]:
i=dData.shape[0]
while i>0:
    i=i-1
    s=dData.loc[i,'Postal Code']    
    idx=geocsv_df.index[geocsv_df['PostalCode']==s].tolist()
    dData.loc[i,'Latitude']=geocsv_df.loc[idx[0],'Latitude']
    dData.loc[i,'Longitude']=geocsv_df.loc[idx[0],'Longitude']
#dData.head(12)

1.put geodata on the map
2. check something
5.
10.

In [8]:
#put geodata on the map

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of toronto using latitude and longitude values
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough,postalcode in zip(dData['Latitude'], dData['Longitude'], dData['Borough'], dData['Postal Code']):
    label = '{}, {}'.format(postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False,
#       width='50%', height='50%'
    ).add_to(map_toronto)  
    

map_toronto

In [9]:
CLIENT_ID = 'DZVJPHGG0GPRF02NL5YMO341MLFPFPJTOQZXR0NJTCD3ZKHI' # your Foursquare ID
CLIENT_SECRET = '22EGVOPH4Q3VNSX25WCYDPXWIBPLTZOBDZ10SAYE2CVGFENJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#torontoData = dData[dData.Borough == 'Downtown Toronto']
torontoData = dData#[dData['Borough'].isin(['Downtown Toronto','Central Toronto','East Toronto','West Toronto'])]
#torontoData = dData[dData['Borough'].isin(['North York','York','East York'])]

torontoData.set_index('Postal Code',inplace=True)
torontoData=torontoData.reset_index()

print(torontoData.shape[0])

torontoData.head()

103


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        LIMIT=100    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues=pd.DataFrame(data=getNearbyVenues(names=torontoData['Postal Code'],
                                   latitudes=dData['Latitude'],
                                   longitudes=dData['Longitude']))
print(toronto_venues.shape)
toronto_venues.head()

(1154, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
2,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
3,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection


In [19]:
#amount of unique categories in dataframe
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 215 uniques categories.


In [20]:
toronto_venues_check=toronto_venues.groupby('Neighborhood').count()
toronto_venues_check.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude','Venue Longitude'], axis=1, inplace=True)
#toronto_venues_check.head(10)
#toronto_venues_check.groupby('Venue').count()

In [21]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

#tor_onehot.head(3)

In [22]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
#tor_grouped

In [23]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

In [26]:
# set number of clusters
kclusters = 4

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 1], dtype=int32)

In [27]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
tor_merged = torontoData

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Postal Code')
tor_merged.groupby('Cluster Labels').count() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2.0,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
3.0,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Postal Code'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)

    if (cluster!=cluster):
        cluster=0

    clstr=int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clstr-1],
        fill=True,
        fill_color=rainbow[clstr-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

lets describe clusters

In [36]:
clstr1=tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]
clstr1
#clstr1.groupby('3rd Most Common Venue').count() 
print ()'not very many dots in this cluster, far from center, coffeshops and exotic restorans')


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,Etobicoke,0.0,Pizza Place,Coffee Shop,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant
72,North York,0.0,Coffee Shop,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant
76,Mississauga,0.0,Coffee Shop,Hotel,American Restaurant,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


In [37]:
# not very many dots in this cluster, also exotic restorans
clstr2=tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]
clstr2


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,East York,1.0,Intersection,Beer Store,Bar,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
50,North York,1.0,Intersection,Furniture / Home Store,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
85,Scarborough,1.0,Intersection,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop


In [38]:
# this one is more interesting - places for free time,  recreation and sport, my favourite cluster:)
clstr3=tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]
clstr3


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Park,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
19,East Toronto,2.0,Park,Playground,Trail,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
21,York,2.0,Women's Store,Park,Gym,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
35,East York,2.0,Park,Intersection,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
52,North York,2.0,Park,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
64,York,2.0,Park,Jewelry Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
66,North York,2.0,Park,Construction & Landscaping,Convenience Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant
77,Etobicoke,2.0,Park,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
83,Central Toronto,2.0,Shopping Mall,Park,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant
91,Downtown Toronto,2.0,Park,Campground,Pool,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop


In [39]:
# this one contains most part of dots, it concentrated in central and downtown parts of toronto different kind of places...
clstr4=tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]
clstr4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Pizza Place,Intersection,Electronics Store,Escape Room,Ethiopian Restaurant,Dim Sum Restaurant,Dumpling Restaurant
2,Downtown Toronto,3.0,Furniture / Home Store,Gym,Gastropub,Breakfast Spot,Spa,Bus Stop,Flower Shop,Food Truck,Distribution Center,Bakery
3,North York,3.0,Clothing Store,Boutique,Accessories Store,Event Space,Coffee Shop,Furniture / Home Store,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,Downtown Toronto,3.0,Coffee Shop,Italian Restaurant,Park,Portuguese Restaurant,Bubble Tea Shop,Sandwich Place,Sushi Restaurant,Restaurant,Café,Thai Restaurant
7,North York,3.0,Tennis Court,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,3.0,Pool,Women's Store,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant,Donut Shop
99,Downtown Toronto,3.0,Gay Bar,Coffee Shop,Burger Joint,Japanese Restaurant,Dessert Shop,Gym,Dance Studio,Martial Arts School,Ethiopian Restaurant,Middle Eastern Restaurant
100,East Toronto,3.0,Auto Workshop,Fast Food Restaurant,Park,Brewery,Farmers Market,Light Rail Station,Cuban Restaurant,Donut Shop,Convenience Store,Cosmetics Shop
101,Etobicoke,3.0,Breakfast Spot,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Dumpling Restaurant
